### Future Climate Models

The Coupled Model Intercomparison Project (CMIP) 6 is a publicly available project under the World Climate Research Programme (WCRP) providing simulations from 120 global climate models and 45 institutions. For this project, the Norwegian Earth System Model (NorESM2) is used due its to its coverage of the four aforementioned ocean parameters influencing corrosion. 

The Representative Concentration Pathway (RCP), developed by the Intergovernmental Panel on Climate Change (IPCC), consists of a set of scenarios predicting the concentrations of greenhouse gases in the atmosphere until 2100.

In [ ]:
import pandas as pd
import os
import xarray as xr

def extract_projection(filename, date, param_OI):
    current_dir = os.getcwd()  # Get the current working directory
    file_path = os.path.join(current_dir, f'data\{filename}')
    ds = xr.open_dataset(file_path, decode_times=False)

    time_values_days = ds['time'].values
    start_date = pd.Timestamp('1800-01-01')
    time_values_normal = [start_date + pd.Timedelta(days=float(days)) for days in time_values_days]
    date_values = [str(dt.date()) for dt in time_values_normal]
    mask_desired_date = [d.startswith(date) for d in date_values]


    if param_OI == 'pH':
        data = ds.sel(time=ds['time'][mask_desired_date], lev=0)
        lat = data.latitude.values
        lon = data.longitude.values
        param = data['ph'].values

    else:
        data = ds.sel(time=ds['time'][mask_desired_date])
        lat = data.latitude.values
        lon = data.longitude.values

        if param_OI == 'temp':
            param = data['tos'].values
        elif param_OI == 'sal':
            param = data['sos'].values
        elif param_OI == 'doxy':
            param = data['o2satos'].values


    return lat, lon, param[0, :, :]



def generate_dfs_proj(filename, date):
    # print(filename)
    if filename.startswith('tos'):
        lat, lon, param_data = extract_projection(filename, date=date, param_OI='temp')
        param = 'temp'
    elif filename.startswith('sos'):
        lat, lon, param_data = extract_projection(filename, date=date, param_OI='sal')
        param = 'sal'
    elif filename.startswith('o2'):
        lat, lon, param_data = extract_projection(filename, date=date, param_OI='doxy')
        param = 'doxy'
    elif filename.startswith('ph'):
        lat, lon, param_data = extract_projection(filename, date=date, param_OI='pH')
        param = 'pH'

    return lat, lon, param_data, param


In [ ]:
from datetime import datetime, timedelta

def days_since_to_date(days_since, ref_date="1800-01-01"):
    reference_date = datetime.strptime(ref_date, "%Y-%m-%d")
    target_date = reference_date + timedelta(days=days_since)
    
    year = target_date.year
    month = target_date.month
    date_str = f'{year}-{month:02}'
    
    return date_str


In [ ]:
import re
import pickle
current_dir = os.getcwd()  # Get the current working directory
filenames = os.listdir(current_dir)
filenames.sort()

# Group names by their starting two characters
grouped_filenames = {}
for filename in filenames:
    prefix = filename[:2]
    if prefix in grouped_filenames:
        grouped_filenames[prefix].append(filename)
    else:
        grouped_filenames[prefix] = [filename]

# Loop through groups 

group_data = {}

for prefix, group in grouped_filenames.items():
    print(f"Group: {prefix}")

    group_dict = {} 

    for filename in group:
        file_path = os.path.join(current_dir, f'data\{filename}')
        ds = xr.open_dataset(file_path, decode_times=False)

        # Convert time values in ds to a list of dates
        time_values = ds["time"].values
        dates = [days_since_to_date(days_since) for days_since in time_values]

        for i in range(len(dates)):
            if prefix == 'ph':
                data = ds.sel(time=ds['time'][i], lev=0)
                lat = data.latitude.values
                lon = data.longitude.values
                param = data['ph'].values

            else:
                data = ds.sel(time=ds['time'][i])
                lat = data.latitude.values
                lon = data.longitude.values

                if prefix == 'to':
                    param = data['tos'].values
                elif prefix == 'so':
                    param = data['sos'].values
                elif prefix == 'o2':
                    param = data['o2satos'].values

            if dates[i] in group_dict:
                group_dict[dates[i]] = pd.DataFrame(param)  
            else:
                group_dict[dates[i]] = pd.DataFrame(param)  

            if i == 0:
                df_lat = pd.DataFrame(lat)
                df_lon = pd.DataFrame(lon)

        group_data[prefix] = group_dict


rows = []

all_dates = set(date for data_dict in group_data.values() for date in data_dict.keys())

for date in all_dates:
    row = {'Date': date}

    for prefix, data_dict in group_data.items():
        if date in data_dict:
            row[prefix] = data_dict[date]
        else:
            row[prefix] = []

    rows.append(row)

entire_df = pd.DataFrame(rows)
entire_df.set_index('Date', inplace=True)

Store the future projection data in the form of a dataframe with the parameter value, another dataframe with the latitudes and a third dataframe with the longitudes.

In [ ]:

pickle_filename = os.path.join(current_dir, "data", "ocean_params_proj.pickle")
with open(pickle_filename, "wb") as file:
    pickle.dump(entire_df, file)

In [ ]:
pickle_filename = os.path.join(current_dir, "data", "ocean_lat_proj.pickle")
with open(pickle_filename, "wb") as file:
    pickle.dump(df_lat, file)

pickle_filename = os.path.join(current_dir, "data", "ocean_lon_proj.pickle")
with open(pickle_filename, "wb") as file:
    pickle.dump(df_lon, file)